In [3]:
import time
import heapq
import numpy as np

from random import choice
from random import randint

from collections import Counter
from collections import defaultdict

In [4]:
#with open("Downloads/test1_new.txt") as f:
 #   lines1 = f.read().splitlines()

with open("test2_new.txt") as f:
    lines2 = f.read().splitlines()

#with open("Downloads/test3_new.txt") as f:
 #   lines3 = f.read().splitlines()
    
with open("test1_aycan.txt") as f:
    lines4 = f.read().splitlines() 
    
    
with open("test_aycan.txt") as f:
    lines5 = f.read().splitlines() 

# Load File

In [5]:
lines=lines2

number_of_vertices = int(lines[0])
number_of_edges = int(lines[1])

vertices = lines[2:2+number_of_vertices]
edges = lines[2+number_of_vertices:]

ids_and_populations = [tuple(map(int, vertices[i].split(" "))) for i in range(len(vertices))]
populations = dict(sorted(dict(ids_and_populations).items())) #redundant sort

mydict = lambda: defaultdict(lambda: defaultdict())
G = mydict()

for i in range(len(edges)):
    source, target, weight = map(int, edges[i].split(" "))
    G[source][target] = weight
    G[target][source] = weight
    
populations

{0: 68,
 1: 169,
 2: 34,
 3: 234,
 4: 15,
 5: 4,
 6: 171,
 7: 213,
 8: 233,
 9: 80,
 10: 30,
 11: 171,
 12: 128,
 13: 125,
 14: 45,
 15: 31,
 16: 46,
 17: 110,
 18: 200,
 19: 215,
 20: 225,
 21: 111,
 22: 101,
 23: 125,
 24: 8,
 25: 196,
 26: 161,
 27: 52,
 28: 80,
 29: 109,
 30: 154,
 31: 71,
 32: 167,
 33: 215,
 34: 51,
 35: 58,
 36: 180,
 37: 74,
 38: 116,
 39: 212,
 40: 245,
 41: 148,
 42: 74,
 43: 245,
 44: 183,
 45: 227,
 46: 45,
 47: 9,
 48: 206,
 49: 107,
 50: 35,
 51: 137,
 52: 17,
 53: 69,
 54: 224,
 55: 104,
 56: 83,
 57: 91,
 58: 141,
 59: 114,
 60: 53,
 61: 218,
 62: 195,
 63: 174,
 64: 158,
 65: 178,
 66: 234,
 67: 136,
 68: 175,
 69: 247,
 70: 224,
 71: 172,
 72: 180,
 73: 67,
 74: 249,
 75: 140,
 76: 104,
 77: 106,
 78: 135,
 79: 47,
 80: 167,
 81: 21,
 82: 223,
 83: 128,
 84: 225,
 85: 2,
 86: 207,
 87: 17,
 88: 242,
 89: 206,
 90: 48,
 91: 15,
 92: 25,
 93: 214,
 94: 173,
 95: 105,
 96: 186,
 97: 48,
 98: 21,
 99: 17,
 100: 190,
 101: 137,
 102: 195,
 103: 95,
 104: 2

# Functions

In [6]:
def select_neighbors(G, sub_graph, current_node, k):
    if k == 0:
        return sub_graph

    for j in G[current_node].items():
        sub_graph[current_node][j[0]] = j[1]
        sub_graph[j[0]][current_node] = j[1]

        sub_graph = select_neighbors(G, sub_graph, j[0], k - 1)
    return sub_graph


def merge_graph(dict1, dict2):
    for key, value in dict2.items():
        for subkey, subvalue in value.items():
            dict1[key][subkey] = subvalue

# INPUT:
# G: dict, populations: dict
def dijkstra(G, populations, source): #FIXME: THIS IS POPULATION_LIST OF OTHERS
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0

    pq = []
    for node in G:
        heapq.heappush(pq, (node, costs[node]))

    while len(pq) != 0:
        current_node, current_node_distance = heapq.heappop(pq)
        for neighbor_node in G[current_node]:
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                costs[neighbor_node] = distance
                heapq.heappush(pq, (neighbor_node, distance))
    
    #return (costs.values(),population_list[])
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    populations_values_lst = list(dict(sorted(populations.items())).values())
    return np.sum(np.array(sorted_costs_lst) * np.array(populations_values_lst))


def random_start(G):
    res = [choice(list(G.keys())), choice(list(G.keys()))]
    if res[0] == res [1]:
        return random_start(G)
    print(f"Random start: {res}")
    return res


#//2 * O((V+E)*logV) = O(E*logV) //
def allocation_cost(G, population_list, i,j):
    return [dijkstra(G,population_list, i),dijkstra(G,population_list, j)]


# 2*O(V)*O(E*logV) = O(E*V*logV) #
def algo(G):
    population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
    #population_list = [populations[i] for i in G.keys()]
    selected_vertices = random_start(G) 
    selected_costs = allocation_cost(G,population_list, selected_vertices[0],selected_vertices[1])
    
    for not_selected in G.keys():
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra(G,population_list, not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                selected_vertices[bigger_index] = not_selected
    return(selected_vertices,selected_costs)


# V times Dijkstra
def sub_graph_apsp(G):
    population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
    selected_vertex = choice(list(G.keys()))
    selected_cost = dijkstra(G,population_list, selected_vertex)
    
    for node in G.keys():
        if node is not selected_vertex:
            this_cost = dijkstra(G, population_list, node) 
            if this_cost < selected_cost:
                selected_cost = this_cost
                selected_vertex = node    
    return selected_vertex, selected_cost


def algorithm_sub_graph_apsp(G, starting_node, k, hop_list):
    sub_graph = lambda: defaultdict(lambda: defaultdict())
    sub_graph = sub_graph()
    sub_graph = select_neighbors(G, sub_graph, current_node=starting_node, k=k)
    next_node, cost = sub_graph_apsp(sub_graph)
    #print(next_node)
    
    if len(hop_list) > 0 and next_node == hop_list[-1][0]:
        return next_node, cost
    
    hop_list.append((next_node, cost))
    return algorithm_sub_graph_apsp(G, next_node, k, hop_list)


def algo2(G, k):
    nodes = []
    costs = []
    
    for i in range(10):
        node, cost = algorithm_sub_graph_apsp(G, choice(list(G.keys())), k, [])
        nodes.append(node)
        costs.append(cost)
        
    counter = Counter(nodes)
    most_commons = counter.most_common(2)
    target_nodes = (most_commons[0][0], most_commons[1][0])
    
    sub_graph1 = lambda: defaultdict(lambda: defaultdict())
    sub_graph1 = sub_graph1()
    sub_graph1 = select_neighbors(G, sub_graph1, target_nodes[0], k=k)
    
    sub_graph2 = lambda: defaultdict(lambda: defaultdict())
    sub_graph2 = sub_graph2()
    sub_graph2 = select_neighbors(G, sub_graph2, target_nodes[1], k=k)

    merge_graph(sub_graph1, sub_graph2)

    return algo(sub_graph1)
    #return target_nodes


In [50]:

#for i in range(20):
#print(algo2(G, k=5))

# ([31470, 715224], [195627566180449, 194597112873142])


(dict_values([582, 603, 411, 379, 291, 310, 362, 450, 238, 116, 126, 1153, 1006, 555, 471, 317, 968, 800, 22, 91, 903, 950, 505, 466, 1204, 1232, 1276, 313, 490, 458, 308, 505, 13, 550, 149, 321, 631, 254, 569, 91, 867, 27, 255, 894, 968, 1024, 869, 287, 334, 460, 283, 823, 605, 535, 572, 179, 871, 969, 337, 166, 1036, 194, 328, 1432, 1330, 931, 361, 212, 619, 810, 835, 902, 39, 673, 551, 253, 310, 989, 968, 237, 1162, 865, 191, 1048, 67, 620, 1077, 196, 376, 535, 1033, 337, 518, 281, 325, 424, 623, 247, 919, 106, 380, 902, 1232, 1267, 294, 617, 177, 1009, 491, 338, 1330, 394, 572, 684, 1223, 811, 459, 705, 712, 765, 539, 719, 1490, 347, 217, 769, 608, 972, 1340, 1255, 250, 915, 906, 418, 132, 931, 435, 539, 1306, 492, 765, 752, 627, 264, 693, 380, 437, 543, 861, 940, 156, 1034, 344, 865, 140, 450, 1210, 655, 915, 927, 966, 606, 684, 651, 650, 532, 327, 665, 1220, 79, 521, 349, 319, 494, 844, 1502, 1270, 890, 336, 582, 768, 171, 472, 218, 325, 1265, 970, 513, 753, 953, 156, 69, 306, 12

In [ ]:
#for i in range(10):
   # print(algo(G))

In [ ]:
#population_list = [populations[i] for i in G.keys()]
#dijkstra(G, population_list, 71)

In [ ]:
'''
sub_graph = lambda: defaultdict(lambda: defaultdict())
sub_graph = sub_graph()
start_time = time.time()
select_neighbors(G, sub_graph, 5000, k=7)
print(time.time() - start_time)

n = len(sub_graph.values()) / 2
n * (n-1) / 2
'''

# Main

In [ ]:
# Initialize the sub-graph
a = dict()
cur_node = 40
for i in range(10):
    sub_graph = lambda: defaultdict(lambda: defaultdict())
    sub_graph = sub_graph()
    sub_graph = select_neighbors(G, sub_graph, current_node=i*4 + 49, k=5)
    hospital_locations, costs = algo(sub_graph)
    
    if hospital_locations[0] not in a:
        a[hospital_locations[0]] = 1
    else:
        a[hospital_locations[0]] += 1
    if hospital_locations[1] not in a:
        a[hospital_locations[1]] = 1
    else:
        a[hospital_locations[1]] += 1
#print(hospital_locations, costs)

keys = np.array([k for k, v in a.items()])
vals = np.array([v for k, v in a.items()])

keys[np.argmax(vals)]

n = 2
indices = (-vals).argsort()[:n]
print(keys[indices[0]])
print(keys[indices[1]])

In [ ]:
#sub_graph = lambda: defaultdict(lambda: defaultdict())
#sub_graph = sub_graph()
#sub_graph = select_neighbors(G, sub_graph, current_node=40, k=4)
#hospital_locations, costs = algo(sub_graph)
#print(hospital_locations, costs)




In [ ]:
#sub_graph = lambda: defaultdict(lambda: defaultdict())
#sub_graph = sub_graph()
#sub_graph = select_neighbors(G, sub_graph, current_node=40, k=5)

#len(list(sub_graph.values()))

In [ ]:
#hospital_locations, costs = algo(G)
#print(hospital_locations, costs)

In [ ]:
#population_list = [populations[i] for i in G.keys()]
#dijkstra(G, population_list, 306690)

In [ ]:
from collections import Counter
a = [1,2,3,1,2,1,1,1,3,2,2,1]
b = Counter(a)
print(b.most_common(2))

# New Heuristic w/ added Local Optimization over 'algo'

In [7]:
# Global/Regional Interchange Algorithm
def GRIA(G,k):
    population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
    selected_vertices = random_start(G) 
    selected_costs = allocation_cost(G,population_list, selected_vertices[0],selected_vertices[1])
    
    for not_selected in G.keys():
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra(G,population_list, not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                
                temp = regional_interchange(G,not_selected,k)
                if temp != min(selected_vertices):
                    selected_vertices[bigger_index] = temp
                # Regional Interchange
                # k-th neighbor -> 5. order neighbor subgraph
                    # V^2*logV 1-hospital problem
    return(selected_vertices,selected_costs)

def regional_interchange(G,current_node,k):
    #k-th neighbor subgraph
    sub_graph = lambda: defaultdict(lambda: defaultdict())
    sub_graph = sub_graph()
    select_neighbors(G, sub_graph, current_node, k)
    #1-hospital
    print(current_node,sub_graph_apsp(sub_graph)[0])
    return sub_graph_apsp(sub_graph)[0]
    

In [8]:
GRIA(G,5)

Random start: [219, 111]
244 214
239 26
240 71
221 214
157 71
125 71
101 101
71 71


([214, 71], [13484179, 13478689])

In [11]:
algo2(G,5)

Random start: [240, 226]


([157, 71], [7837472, 7834750])

In [10]:
algo(G)

Random start: [17, 30]


([157, 71], [13484179, 13478689])

In [12]:
def dijkstra_path(G, population_list, source):
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0
    #display(source,costs)

    pq = []
    for node in G:
        heapq.heappush(pq, (node, costs[node]))

    while len(pq) != 0:
        current_node, current_node_distance = heapq.heappop(pq)
        for neighbor_node in G[current_node]:
            #print(current_node,costs[source])
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                #if source==neighbor_node:
                    #print('here')
                costs[neighbor_node] = distance
                heapq.heappush(pq, (neighbor_node, distance))
    
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    sorted_populations_lst = list(dict(sorted(population_list.items())).values())
    #print(np.array(sorted_costs_lst) ,np.array(sorted_populations_lst))
    return np.array(sorted_costs_lst) * np.array(sorted_populations_lst)
    #return list(dict(sorted(costs.items())).values())

In [111]:
population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
dijkstra_path(G, population_list, 15)

array([  0,  11,  22,  45,  28,  55,  48,  91,  32, 144, 100, 110,  72,
        78,  98,   0, 224])

In [44]:
population_list = [populations[i] for i in G.keys()]
#temp = dijkstra_path(G, population_list,200)
#[e[1] for e in sorted(list(temp.items()))]
#sorted(list(temp.items()))
G.keys()

dict_keys([244, 246, 239, 240, 238, 245, 235, 233, 232, 248, 231, 229, 249, 228, 241, 226, 223, 242, 222, 225, 220, 247, 219, 221, 218, 224, 227, 217, 216, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 237, 198, 197, 230, 196, 195, 194, 193, 243, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 215, 182, 181, 180, 179, 178, 236, 177, 176, 175, 174, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162, 161, 160, 159, 234, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 140, 139, 138, 137, 136, 135, 141, 134, 133, 132, 131, 130, 129, 128, 173, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 127, 88, 87, 111, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40,

In [13]:
#APSP matrix
#def dijkstra(G, population_list, source):


APSP = np.zeros((number_of_vertices,number_of_vertices))
population_list = dict(sorted([(k, populations[k]) for k in G.keys()]))
for vertex in vertices:
    vertex= int(vertex.split()[0])
    APSP[vertex] = [e for e in dijkstra_path(G, population_list,vertex)]

def V4(APSP):
    n = len(APSP)

    res = {}

    temp_arr = APSP.copy()

    count=0
    for first in range(n):
        for second in range(n):
            if first==second:
                continue
            count+=1
            #print(first,second)
            temp_arr = APSP.copy()
            for row in temp_arr:
                if row[first]<row[second]:
                    row[second]=0
                else:
                    row[first]=0 
            #print(temp_arr,count)
            temp = sum(sum(temp_arr)) 
            #if temp in res.keys():
                #print(res[sum(sum(temp_arr))])
                #print('here!!!!!!!! ' ,first,second)
            #print(temp,(first,second))
            res[temp]=(first,second) #FIXME: there are collisions but is it a problem?

            

    ret=res[min(res.keys())]
    display(len(res))
    #display(res)
    print('pick {}th and {}th vertices to place hospitals!'.format(ret[0],ret[1]))
    return ret

#print(APSP)
V4(APSP)


31100

pick 74th and 66th vertices to place hospitals!


(74, 66)

In [ ]:
APSP = np.zeros((2,2))
APSP[0]

In [100]:
# 8 13
# 9 16

display(sum(dijkstra_path(G, population_list, 8))+sum(dijkstra_path(G, population_list, 13)))
display(sum(dijkstra_path(G, population_list, 14))+sum(dijkstra_path(G, population_list, 16)))

2060

2647